# Importing Libraries

In [7]:
# loading the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Loading Data

In [22]:
df = pd.read_csv('baseline_df.csv')

In [23]:
date1 = pd.to_datetime(df['time:timestamp'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f%z')
date2 = pd.to_datetime(df['time:timestamp'], errors='coerce', format='%Y-%m-%d %H:%M:%S%z')
df['date'] = date1.fillna(date2)

In [24]:
df["date_milliseconds"] = (df['date'] - pd.Timestamp("1970-01-01", tz = "UTC")) // pd.Timedelta('1ms')

In [25]:
df["duration"] = df.groupby("case:concept:name")["date_milliseconds"].shift(-1) - df["date_milliseconds"]

In [26]:
df.head(10)

,org:resource,concept:name,time:timestamp,case:concept:name,concept:name_eng,position,next_activity,date,date_milliseconds,duration
0,112.0,A_SUBMITTED,2011-10-01 00:38:44.546000+00:00,173688,A_SUBMITTED,1,A_PARTLYSUBMITTED,2011-10-01 00:38:44.546000+00:00,1317429524546,334.0
1,112.0,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+00:00,173688,A_PARTLYSUBMITTED,2,A_PREACCEPTED,2011-10-01 00:38:44.880000+00:00,1317429524880,53026.0
2,112.0,A_PREACCEPTED,2011-10-01 00:39:37.906000+00:00,173688,A_PREACCEPTED,3,W_Completeren aanvraag,2011-10-01 00:39:37.906000+00:00,1317429577906,969.0
3,112.0,W_Completeren aanvraag,2011-10-01 00:39:38.875000+00:00,173688,W_Complete request,4,W_Completeren aanvraag,2011-10-01 00:39:38.875000+00:00,1317429578875,39427562.0
4,NaN,W_Completeren aanvraag,2011-10-01 11:36:46.437000+00:00,173688,W_Complete request,5,A_ACCEPTED,2011-10-01 11:36:46.437000+00:00,1317469006437,356871.0
5,10862.0,A_ACCEPTED,2011-10-01 11:42:43.308000+00:00,173688,A_ACCEPTED,6,O_SELECTED,2011-10-01 11:42:43.308000+00:00,1317469363308,145935.0
6,10862.0,O_SELECTED,2011-10-01 11:45:09.243000+00:00,173688,O_SELECTED,7,A_FINALIZED,2011-10-01 11:45:09.243000+00:00,1317469509243,0.0
7,10862.0,A_FINALIZED,2011-10-01 11:45:09.243000+00:00,173688,A_FINALIZED,8,O_CREATED,2011-10-01 11:45:09.243000+00:00,1317469509243,1954.0
8,10862.0,O_CREATED,2011-10-01 11:45:11.197000+00:00,173688,O_CREATED,9,O_SENT,2011-10-01 11:45:11.197000+00:00,1317469511197,183.0
9,10862.0,O_SENT,2011-10-01 11:45:11.380000+00:00,173688,O_SENT,10,W_Nabellen offertes,2011-10-01 11:45:11.380000+00:00,1317469511380,174.0


# Assigning the most frequent next event in each position

In [13]:
# Finidng the most frequent next activity for each position
position_dict = {}
for j in range(1, max(df["position"]+1)):
    dic = {}
    for i in (df[df['position'] == j]).index:
        if df['next_activity'][i] in dic:
            dic[df['next_activity'][i]] += 1
        else:
            dic[df['next_activity'][i]] = 1
    
    position_dict[j] = max(dic, key=dic.get)

In [14]:
# Iterating through the dataframe and map the values of position_dict to the column predicted_next_activity with lambda
df['predicted_next_activity'] = df['position'].map(lambda x: position_dict[x])

In [16]:
# printing the accuracy, precision, recall and f1-score of the model separately without report
accuracy = accuracy_score(df["next_activity"], df["predicted_next_activity"])
precision = precision_score(df["next_activity"], df["predicted_next_activity"], average='micro')
recall = recall_score(df["next_activity"], df["predicted_next_activity"], average='micro')
f1 = f1_score(df["next_activity"], df["predicted_next_activity"], average='micro')

f'Accuracy: {accuracy} Precision: {precision} Recall: {recall} F1: {f1}'

'Accuracy: 0.4316933638443936 Precision: 0.4316933638443936 Recall: 0.4316933638443936 F1: 0.4316933638443936'

# Assigning the timestamp for each position

In [30]:
# Finidng the most frequent next activity for each position
position_dict_1 = {}
for j in range(1, 176):
    n = 0
    x = 0
    for i in (df[df['position'] == j]).index:
        if df["duration"][i] > 0 :
            n = n + df["duration"][i]
            x = x + 1
    if x > 0:
        position_dict_1[j] = n/x


In [32]:
df["predicted_timestamp"] = df["position"].map(lambda x: position_dict_1[x] + df["date_milliseconds"])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Aahaan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-83da61098434>", line 1, in <module>
    df["predicted_timestamp"] = df["position"].map(lambda x: position_dict_1[x] + df["date_milliseconds"])
  File "C:\Users\Aahaan\anaconda3\lib\site-packages\pandas\core\series.py", line 4397, in map
    new_values = self._map_values(arg, na_action=na_action)
  File "C:\Users\Aahaan\anaconda3\lib\site-packages\pandas\core\base.py", line 924, in _map_values
    new_values = map_f(values, mapper)
  File "pandas\_libs\lib.pyx", line 2834, in pandas._libs.lib.map_infer
  File "<ipython-input-32-83da61098434>", line 1, in <lambda>
    df["predicted_timestamp"] = df["position"].map(lambda x: position_dict_1[x] + df["date_milliseconds"])
  File "C:\Users\Aahaan\anaconda3\lib\site-packages\pandas\core\generic.py", line 2016, in __array_

TypeError: object of type 'NoneType' has no len()